Build a regression model.

In [11]:
#imports
import numpy as np
import pandas as pd
import sqlite3
import statsmodels.api as sm

In [113]:
#import data
db_connection = sqlite3.connect("python_project.db")

data_df = pd.read_sql_query("SELECT * FROM restaurants", db_connection)

data_df.head()

# do a forward selection ; does distance, price, rating, total_rating have an impact on number of bikes?

,ID,latitude,longitude,free_bikes,distance,name,price,rating,address,total_ratings
0,0,49.262487,-123.114397,8.0,96.0,Marulilu Cafe,2.0,8.2,"451W Broadway W (at Cambie St), Vancouver BC V...",112.0
1,1,49.262487,-123.114397,8.0,299.0,Aperture Coffee Bar,1.0,8.7,"243 West Broadway (at Alberta St), Vancouver B...",94.0
2,3,49.262487,-123.114397,8.0,430.0,Milano Coffee,2.0,9.0,"156 8th Ave W (btwn Columbia & Manitoba St), V...",121.0
3,5,49.262487,-123.114397,8.0,225.0,Elysian Coffee Broadway,1.0,8.2,"590 Broadway W (at Ash St), Vancouver BC V5Z 1E9",199.0
4,8,49.262487,-123.114397,8.0,724.0,Elysian Coffee,1.0,9.1,"2301 Ontario St (at 7th Ave), Vancouver BC V5T...",75.0


In [115]:
data_df.isnull().sum() # 848 nulls for price, 16 for rating, and 16 for total_rating
#Due to time constraints I will remove the NaN as I have over 5000 results

data_df = data_df.dropna()
data_df.isnull().sum() # check
data_df.info() # still have over 4500 results

#drop irrelevent columns
data_df = data_df.drop(columns=['ID', 'latitude', 'longitude', 'name', 'address'])
data_df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4597 entries, 0 to 5455
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             4597 non-null   int64  
 1   latitude       4597 non-null   float64
 2   longitude      4597 non-null   float64
 3   free_bikes     4597 non-null   float64
 4   distance       4597 non-null   float64
 5   name           4597 non-null   object 
 6   price          4597 non-null   float64
 7   rating         4597 non-null   float64
 8   address        4597 non-null   object 
 9   total_ratings  4597 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 395.1+ KB


,free_bikes,distance,price,rating,total_ratings
0,8.0,96.0,2.0,8.2,112.0
1,8.0,299.0,1.0,8.7,94.0
2,8.0,430.0,2.0,9.0,121.0
3,8.0,225.0,1.0,8.2,199.0
4,8.0,724.0,1.0,9.1,75.0


In [93]:
Y_var = data_df['free_bikes']
my_data = data_df.drop(columns=['free_bikes'])
X_list = []

for column in my_data:
    X_list.append(sm.add_constant(my_data[column]))

X_list = [sm.add_constant(my_data[column]) for column in my_data.columns]

X_list[1]



,const,price
0,1.0,2.0
1,1.0,1.0
2,1.0,2.0
3,1.0,1.0
4,1.0,1.0
...,...,...
5451,1.0,2.0
5452,1.0,2.0
5453,1.0,2.0
5454,1.0,2.0


In [94]:
num_bikes_model = sm.OLS(Y_var, X_list[0])
num_bikes_results = num_bikes_model.fit()
num_bikes_ajd_R2 = num_bikes_results.rsquared_adj
num_bikes_pvalue = num_bikes_results.pvalues

print(num_bikes_ajd_R2, num_bikes_pvalue)

0.00636665493615951 const       0.000000e+00
distance    3.616575e-08
dtype: float64


In [95]:
my_models = [sm.OLS(Y_var, X) for X in X_list]
my_results = [model.fit() for model in my_models]
my_adj_R2 = [result.rsquared_adj for result in my_results]
my_pval = [result.pvalues for result in my_results]

In [96]:
for i in range(len(my_adj_R2)):
    print(f"adj_R2 : {my_adj_R2[i]}, p-value: {*my_pval[i],} column: {my_data.columns[i]}")

adj_R2 : 0.00636665493615951, p-value: (0.0, 3.6165752648205854e-08) column: distance
adj_R2 : 0.002313402272376197, p-value: (9.871333066513711e-168, 0.0006451311503806568) column: price
adj_R2 : 0.0013292952429835303, p-value: (1.2055878679623316e-18, 0.007659986281584407) column: rating
adj_R2 : 0.00831437861364892, p-value: (0.0, 3.5246306512662444e-10) column: total_ratings


Total ratings appears to have the highest R2

In [97]:
leftover_var = my_data.drop(columns=['total_ratings'], axis=1)
included_var = my_data[['total_ratings']]
X_list = [sm.add_constant(pd.merge(included_var, leftover_var[column], right_index = True, left_index = True)) for column in leftover_var.columns]


In [98]:
my_models = [sm.OLS(Y_var, X) for X in X_list]
my_results = [model.fit() for model in my_models]
my_adj_R2 = [result.rsquared_adj for result in my_results]
my_pval = [result.pvalues for result in my_results]

for i in range(len(my_adj_R2)):
    print(f"adj_R2 : {my_adj_R2[i]}, p-value: {*my_pval[i],} column: {leftover_var.columns[i]}")

adj_R2 : 0.014891172928719554, p-value: (0.0, 1.8891449400358595e-10, 1.9287051588923353e-08) column: distance
adj_R2 : 0.008804291484203364, p-value: (1.0207303120917093e-164, 2.603709704358022e-08, 0.0705737301845264) column: price
adj_R2 : 0.009086824878443922, p-value: (3.577013652476474e-18, 1.295054742119653e-09, 0.03236312822220999) column: rating


Distance has the highest adj R2 value

In [99]:
leftover_var = my_data.drop(columns=['distance'], axis=1)
included_var = my_data[['distance']]
X_list = [sm.add_constant(pd.merge(included_var, leftover_var[column], right_index = True, left_index = True)) for column in leftover_var.columns]


In [100]:
my_models = [sm.OLS(Y_var, X) for X in X_list]
my_results = [model.fit() for model in my_models]
my_adj_R2 = [result.rsquared_adj for result in my_results]
my_pval = [result.pvalues for result in my_results]

for i in range(len(my_adj_R2)):
    print(f"adj_R2 : {my_adj_R2[i]}, p-value: {*my_pval[i],} column: {leftover_var.columns[i]}")

adj_R2 : 0.009116877106157495, p-value: (2.4279732597041414e-158, 1.2348124527546839e-08, 0.00021088741382054992) column: price
adj_R2 : 0.007461370842445136, p-value: (3.7863131497420057e-23, 6.22548730111034e-08, 0.013801405697980757) column: rating
adj_R2 : 0.014891172928719554, p-value: (0.0, 1.9287051588922648e-08, 1.889144940036045e-10) column: total_ratings


Provide model output and an interpretation of the results. 

In [111]:
#Forward selection results
y_var = data_df['free_bikes']
x_var = data_df[['total_ratings', 'distance']]

my_model = sm.OLS(y_var, x_var)
results = my_model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:             free_bikes   R-squared (uncentered):                   0.572
Model:                            OLS   Adj. R-squared (uncentered):              0.572
Method:                 Least Squares   F-statistic:                              3068.
Date:                Sat, 02 Sep 2023   Prob (F-statistic):                        0.00
Time:                        17:09:14   Log-Likelihood:                         -14918.
No. Observations:                4597   AIC:                                  2.984e+04
Df Residuals:                    4595   BIC:                                  2.985e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

The model explains only 57.2% of the variations. The independent variable coefficients are essentially negligable. The p-value indicates the variables are statistically significant.

In [120]:
#backward selection
y_var = data_df['free_bikes']
x_var = data_df.drop('free_bikes', axis=1)
x_var = sm.add_constant(x_var)

bw_model = sm.OLS(y_var, x_var)
bw_results = bw_model.fit()
print(bw_results.summary())

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     20.06
Date:                Sat, 02 Sep 2023   Prob (F-statistic):           2.14e-16
Time:                        18:56:10   Log-Likelihood:                -14089.
No. Observations:                4597   AIC:                         2.819e+04
Df Residuals:                    4592   BIC:                         2.822e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             6.3945      0.754      8.485

What a weird result.... distance and total_ratings have 0 p-values while price and rating are still below the 0.05 threshold. I am going to remove rating to see what happens

In [121]:
#removed price
y_var = data_df['free_bikes']
x_var = data_df.drop(['free_bikes', 'price'], axis=1)
x_var = sm.add_constant(x_var)

bw_model = sm.OLS(y_var, x_var)
bw_results = bw_model.fit()
print(bw_results.summary())

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     25.07
Date:                Sat, 02 Sep 2023   Prob (F-statistic):           4.39e-16
Time:                        19:07:12   Log-Likelihood:                -14092.
No. Observations:                4597   AIC:                         2.819e+04
Df Residuals:                    4593   BIC:                         2.822e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             6.9798      0.707      9.875

The adjusted R2 barely moved, but the p-value for rating did go up to 0.055 which exceeds the 0.05 threshold. I will remove that too and look at the results.

In [122]:
#removed price and rating
y_var = data_df['free_bikes']
x_var = data_df.drop(['free_bikes', 'price', 'rating'], axis=1)
x_var = sm.add_constant(x_var)

bw_model = sm.OLS(y_var, x_var)
bw_results = bw_model.fit()
print(bw_results.summary())

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     35.74
Date:                Sat, 02 Sep 2023   Prob (F-statistic):           3.97e-16
Time:                        19:09:11   Log-Likelihood:                -14094.
No. Observations:                4597   AIC:                         2.819e+04
Df Residuals:                    4594   BIC:                         2.821e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             8.2908      0.186     44.525

# Stretch

How can you turn the regression model into a classification model?